In [1]:
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from glob import glob
from ast import literal_eval
import jsonlines
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import plot_confusion_matrix
import random
from sklearn.utils import shuffle
import torch
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.text import Tokenizer
seed = 2022

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import fasttext

# train word vector
model = fasttext.train_unsupervised('/kaggle/input/allprivacy/all.txt', model='skipgram', dim=300)

# 获取词汇表中的单词列表
words = model.words

# 获取每个单词的词向量
vectors = []
for word in words:
    vector = model[word]
    vectors.append(vector)

# 将词向量转换为PyTorch张量
weights = torch.FloatTensor(vectors)

Read 1M words
Number of words:  6162
Number of labels: 0
Progress: 100.0% words/sec/thread:   45147 lr:  0.000000 avg.loss:  2.092068 ETA:   0h 0m 0s
/tmp/ipykernel_32/2386768707.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  weights = torch.FloatTensor(vectors)


In [5]:
model.save_model('/kaggle/working/fasttext')

In [3]:
weights.shape

torch.Size([6162, 300])

In [19]:
weights

torch.Size([6162, 100])

In [2]:
train = pd.read_csv('/kaggle/input/opp-multilabel-9class/allclass/trainAll.csv')
valid = pd.read_csv('/kaggle/input/opp-multilabel-9class/allclass/validationALL.csv')
test = pd.read_csv('/kaggle/input/opp-multilabel-9class/allclass/testALL.csv')

def str2int(df):
    for col in df.columns[1:]:
#         print(col)
        df[col] = df[col].apply(lambda x: 1 if x == col else 0)
    return df

train = str2int(train)
valid = str2int(valid)
test = str2int(test)

In [4]:
test_model = fasttext.train_unsupervised('/kaggle/input/4fasttext-vec/opptext.txt', model='skipgram', dim=300)

Read 0M words
Number of words:  3607
Number of labels: 0
Progress: 100.0% words/sec/thread:   34001 lr:  0.000000 avg.loss:  2.412618 ETA:   0h 0m 0s


In [ ]:
words = model.words
words

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:

#     def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, num_classes, dense_size):
#         super(MultiLabelCNN, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.convs = nn.ModuleList([nn.Conv2d(1, num_filters, (f, embedding_dim)) for f in filter_sizes])
# #         self.fc = nn.Linear(len(filter_sizes) * num_filters, num_classes)
#         self.fc1 = nn.Linear(len(filter_sizes) * num_filters, dense_size)
#         self.fc2 = nn.Linear(dense_size, num_classes)
        

#     def forward(self, x):
#         x = self.embedding(x) # [batch_size, seq_len, embedding_dim]
#         x = x.unsqueeze(1) # [batch_size, 1, seq_len, embedding_dim]
#         x = [F.relu(conv(x)).squeeze(3) for conv in self.convs] # [batch_size, num_filters, seq_len] * len(filter_sizes)
#         x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x] # [batch_size, num_filters] * len(filter_sizes)
#         x = torch.cat(x, 1) # [batch_size, num_filters * len(filter_sizes)]
#         logits = self.fc(x) # [batch_size, num_classes]
#         output = torch.sigmoid(logits)
#         return output
class MultiLabelCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, num_classes, dense_size):
        super(MultiLabelCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim,padding_idx = 0)
#         self.embedding = nn.Embedding.from_pretrained(embeddings=pretrained_weights,freeze=True)
        self.convs = nn.ModuleList([nn.Conv2d(1, num_filters, (f, embedding_dim)) for f in filter_sizes])
        self.relu = nn.ReLU()
        self.drop_out = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(len(filter_sizes) * num_filters, dense_size)
        self.fc2 = nn.Linear(dense_size, num_classes)


    def forward(self, x):
        x = self.embedding(x) # [batch_size, seq_len, embedding_dim]
        x = x.unsqueeze(1) # [batch_size, 1, seq_len, embedding_dim]
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs] # [batch_size, num_filters, seq_len] * len(filter_sizes)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x] # [batch_size, num_filters] * len(filter_sizes)
        x = torch.cat(x, 1) # [batch_size, num_filters * len(filter_sizes)]
        x = F.relu(self.fc1(x)) # [batch_size, dense_size]
        x = self.drop_out(x)
        logits = self.fc2(x) # [batch_size, num_classes]
        
        output = torch.sigmoid(logits)
        return output
    def load_pretrained_embeddings(self, weights_matrix):
        self.embedding = self.embedding.from_pretrained(weights_matrix,freeze = True)
            
    def fit(self, x_train, y_train, x_val, y_val, batch_size=40, num_epochs=10, learning_rate=0.1):
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
        for epoch in range(num_epochs):
            model.train()
            permutation = torch.randperm(x_train.size()[0]).to(device)
            for i in range(0,x_train.size()[0], batch_size):
                indices = permutation[i:i+batch_size]
                batch_x, batch_y = x_train[indices], y_train[indices]
                y_pred = self(batch_x)
                loss = criterion(y_pred, batch_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            model.eval()
            val_loss = criterion(self(x_val), y_val)
            print(f'Epoch {epoch + 1}/{num_epochs}: Loss = {loss.item():.4f}, Val Loss = {val_loss.item():.4f}')

In [6]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

train_text = train['text'].values
train_tokens = [tokenizer.tokenize(t) for t in train_text]

valid_text = valid['text'].values
valid_tokens = [tokenizer.tokenize(t) for t in valid_text]

test_text = test['text'].values
test_tokens = [tokenizer.tokenize(t) for t in test_text]

In [7]:
whole_tokens = train_tokens + valid_tokens + test_tokens

In [8]:
# 构建词汇表
word2index = {word: index for index, word in enumerate(words)}

In [ ]:
def 

In [10]:
#fast-text vector
X_train = [[word2index.get(token.lower()) for token in tokens if token.lower() in word2index] for tokens in train_tokens]
X_train = [torch.tensor(tokens) for tokens in X_train]
X_train = pad_sequence(X_train, batch_first=True, padding_value=0)

X_valid = [[word2index.get(token.lower()) for token in tokens if token.lower() in word2index] for tokens in valid_tokens]
X_valid = [torch.tensor(tokens) for tokens in X_valid]
X_valid = pad_sequence(X_valid, batch_first=True, padding_value=0)

X_test = [[word2index.get(token.lower()) for token in tokens if token.lower() in word2index] for tokens in test_tokens]
X_test = [torch.tensor(tokens) for tokens in X_test]
X_test = pad_sequence(X_test, batch_first=True, padding_value=0)

In [8]:
#用自带的word2vec
# 构建词汇表
tokenizer = Tokenizer()
tokenizer.fit_on_texts(whole_tokens)

# 将tokens转换为整数索引
X_train = [torch.tensor(tokenizer.texts_to_sequences([t])[0]) for t in train_tokens]

# 对齐序列长度
X_train = pad_sequence(X_train, batch_first=True, padding_value=0)

X_valid = [torch.tensor(tokenizer.texts_to_sequences([t])[0]) for t in valid_tokens]
X_valid = pad_sequence(X_valid, batch_first=True, padding_value=0)

X_test = [torch.tensor(tokenizer.texts_to_sequences([t])[0]) for t in test_tokens]
X_test = pad_sequence(X_test, batch_first=True, padding_value=0)

In [8]:
X_test = [torch.tensor(tokenizer.texts_to_sequences([t])[0]) for t in test_tokens]
X_test = pad_sequence(X_test, batch_first=True, padding_value=0)

In [9]:
X_train

tensor([[  31,   48, 1505,  ...,    0,    0,    0],
        [   5,   16, 1505,  ...,    0,    0,    0],
        [ 446,   32,    6,  ...,    0,    0,    0],
        ...,
        [ 122,  629,    3,  ...,    0,    0,    0],
        [1000,    9,   34,  ...,    0,    0,    0],
        [  36,  399,    7,  ...,    0,    0,    0]])

In [12]:
vocab_size = len(words) + 1
# vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 300
num_filters = 200
filter_sizes = [3, 3, 3]
dense_size = 100
batch_size = 40
num_epochs = 200

In [13]:
# 加载标签数据
y_train = train[[x for x in train.columns[1:]]].values
num_classes = y_train.shape[1]
y_valid = valid[[x for x in valid.columns[1:]]].values

In [14]:
y_train = torch.tensor(y_train).float()
y_valid = torch.tensor(y_valid).float()

In [16]:
# 初始化模型
model = MultiLabelCNN(vocab_size, embedding_dim, num_filters, filter_sizes, num_classes,dense_size)
model.load_pretrained_embeddings(weights)
model = model.to(device)
print(model)
# 训练模型
model.fit(X_train, y_train,x_val=X_valid, y_val=y_valid, num_epochs = num_epochs)

MultiLabelCNN(
  (embedding): Embedding(6162, 300)
  (convs): ModuleList(
    (0-2): 3 x Conv2d(1, 200, kernel_size=(3, 300), stride=(1, 1))
  )
  (relu): ReLU()
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=600, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=9, bias=True)
)
Epoch 1/200: Loss = 0.3775, Val Loss = 0.3699
Epoch 2/200: Loss = 0.3501, Val Loss = 0.3656
Epoch 3/200: Loss = 0.3619, Val Loss = 0.3643
Epoch 4/200: Loss = 0.3794, Val Loss = 0.3639
Epoch 5/200: Loss = 0.3960, Val Loss = 0.3634
Epoch 6/200: Loss = 0.3265, Val Loss = 0.3625
Epoch 7/200: Loss = 0.3694, Val Loss = 0.3619
Epoch 8/200: Loss = 0.3291, Val Loss = 0.3609
Epoch 9/200: Loss = 0.3907, Val Loss = 0.3605
Epoch 10/200: Loss = 0.3712, Val Loss = 0.3601
Epoch 11/200: Loss = 0.3191, Val Loss = 0.3589
Epoch 12/200: Loss = 0.3344, Val Loss = 0.3581
Epoch 13/200: Loss = 0.3405, Val Loss = 0.3574
Epoch 14/200: Loss = 0.3505, Val Loss = 0.3562
Epoch 15/200: Loss = 

In [17]:
y_test = test[[x for x in valid.columns[1:]]].values
y_test = torch.tensor(y_test).float()

In [28]:
model.eval()  # 将模型切换到评估模式，禁用dropout
X_test = X_test.to(device)
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred > 0.5

In [29]:
y_pred = y_pred.tolist()

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.26      0.40        34
           1       0.89      0.67      0.76        81
           2       1.00      0.20      0.33         5
           3       0.84      0.80      0.82       317
           4       0.92      0.83      0.87        69
           5       1.00      0.62      0.76        34
           6       0.86      0.77      0.81       233
           7       0.87      0.72      0.79        46
           8       0.85      0.55      0.66       130

   micro avg       0.86      0.71      0.78       949
   macro avg       0.89      0.60      0.69       949
weighted avg       0.86      0.71      0.77       949
 samples avg       0.85      0.78      0.79       949



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
from sklearn.metrics import multilabel_confusion_matrix
confusion_matrix = multilabel_confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[[620   0]
  [ 34   0]]

 [[573   0]
  [ 81   0]]

 [[649   0]
  [  5   0]]

 [[241  96]
  [ 97 220]]

 [[585   0]
  [ 69   0]]

 [[620   0]
  [ 34   0]]

 [[421   0]
  [233   0]]

 [[608   0]
  [ 46   0]]

 [[524   0]
  [130   0]]]


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')